In [1]:
import torch
from torch import nn
from torchvision import transforms as T
from PIL import Image
import numpy as np
from pathlib import Path
from collections import deque
import random, datetime, os, copy
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
%reload_ext autoreload
%autoreload 2
import gym
from gym.spaces import Box
from gym.wrappers import FrameStack
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
import gym_super_mario_bros.actions as JoypadActions

from lib.env_wrappers import EnvWrapperFactory
from agents.ForgetfulAgent import ForgetfulAgent
from lib.MetricLogger import MetricLogger


In [13]:
imageShape = (84, 84)
actionShape = len(JoypadActions.SIMPLE_MOVEMENT)
net_name = "CNN84x84"

env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, JoypadActions.SIMPLE_MOVEMENT)
env = EnvWrapperFactory.convert(env, shape=imageShape)

agent = ForgetfulAgent(state_shape=env.observation_space.shape, action_shape=actionShape, device=device, net_name=net_name)
agent.load("ForgetfulAgent-CNN84x84-checkpoint-20.pytorch")
print("exploration rate:", agent.exploration_rate)

shape before any transformations: (240, 256, 3)
shape after grayscaler: (240, 256)
shape after resizer: (84, 84)
shape after all transformations: (5, 84, 84)
exploration rate: 0.9849459584914215


In [11]:

done = True
count = 0 
for step in range(1000000):
    if done:
        count += 1
        if count > 2:
            break
        state = env.reset()
    state, reward, done, info = env.step(agent.getAction(state))
    env.render()
env.close()